In [35]:
import pandas as pd
import numpy as np
import math as m


from numba import jit, njit, prange

%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [91]:
dir_path = 'data/waypoints.csv'
df = pd.read_csv(dir_path)
wx = df['y'].values
wy = df['x'].values
wx = wx[20:50]
wy = wy[20:50]

# Vectorisation

In [37]:
def solve_1st_derivative(x, y):

    dx = np.ediff1d(x)
    dy = np.ediff1d(y)
    dx = np.concatenate((dx, [dx[0]]))
    dy = np.concatenate((dy, [dy[0]]))

    return dx, dy

In [38]:
%%capture

def calculate_yaw_vectorised(x, y):

    dx, dy = solve_1st_derivative(x, y)
        
    return np.arctan2(dy, dx)

def calculate_yaw_optimised(x, y):

    dx, dy = solve_1st_derivative(x, y)

    yaw = []

    yaw.append(m.atan2(dy[0], dx[0]))

    for i in range(1, len(x) - 1):
        d0 = np.array([dx[i], dy[i]])
        d0_hat = d0/np.linalg.norm(d0)

        d1 = np.array([dx[i-1], dy[i-1]])
        d1_hat = d1/np.linalg.norm(d1)

        d_bisect = d0_hat + d1_hat
        
        yaw.append(m.atan2(d_bisect[1], d_bisect[0]))

    yaw.append(m.atan2(dy[-1], dx[-1]))
        
    return yaw

def calculate_yaw(x, y):

    dx, dy = solve_1st_derivative(x, y)

    yaw = []

    for i in range(0, len(x)):
        yaw.append(np.arctan2(dy[i], dx[i]))

    return yaw

t_vectorised = %timeit -o calculate_yaw_vectorised(wx, wy)
t_optimised = %timeit -o calculate_yaw_optimised(wx, wy)
t_default = %timeit -o calculate_yaw(wx, wy)

In [39]:
print("\nVectorised test:\n{}".format(t_vectorised))
print("\n'Optimised' test:\n{}".format(t_optimised))
print("\nDefault test:\n{}".format(t_default))


Vectorised test:
7.61 µs ± 34.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

'Optimised' test:
345 µs ± 8.87 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

Default test:
52.4 µs ± 1.84 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Exponents

In [40]:
%%capture

def nat_pow(base, exponent):
    result = 1.0

    for i in range(0, exponent):
        result *= base

    return result

a = 7.6
exp = 3
t_manual = %timeit -o a*a
t_numpy = %timeit -o np.power(a, exp)
t_math = %timeit -o m.pow(a, exp)
t_optimised = %timeit -o nat_pow(a, exp)
t_standard = %timeit -o pow(a, exp)
t_default = %timeit -o a**exp

In [41]:
print("\nManual test:\n{}".format(t_manual))
print("\nNumpy test:\n{}".format(t_numpy))
print("\nMath test:\n{}".format(t_math))
print("\n'Optimised' test:\n{}".format(t_optimised))
print("\nStandard test:\n{}".format(t_standard))
print("\nDefault test:\n{}".format(t_default))


Manual test:
27.9 ns ± 1.27 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)

Numpy test:
1.12 µs ± 7.38 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

Math test:
101 ns ± 0.503 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)

'Optimised' test:
227 ns ± 3.32 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

Standard test:
81.4 ns ± 0.704 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)

Default test:
62 ns ± 0.203 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


# Path Curvature

In [42]:
%%capture

from libs.cubic_spline_interpolator import calculate_spline_curvature, generate_cubic_path

def calculate_curvature_vector(wx, wy, ds):

    x, y = generate_cubic_path(wx, wy, ds)
    dp = solve_1st_derivative(x, y)
    delta_s = np.hypot(dp[0], dp[1])
    u_i = dp / delta_s

    du = solve_1st_derivative(u_i[0], u_i[1])
    abs_du = np.hypot(du[0], du[1])
    k_i = abs_du / delta_s

    return k_i

def calculate_curvature(wx, wy, ds):

    x, y = generate_cubic_path(wx, wy, ds)
    dx, dy = solve_1st_derivative(x, y)
    ddx, ddy = solve_1st_derivative(dx, dy)

    return (ddy*dx - ddx*dy) / ((dx*dx + dy*dy)**1.5)

ds = 1.0

t_vector = %timeit -o calculate_curvature_vector(wx, wy, ds)
r_vector = (1 / calculate_curvature_vector(wx, wy, ds)).tolist()[:10]

t_spline = %timeit -o calculate_spline_curvature(wx, wy, ds)
r_spline = (1 / np.array(calculate_spline_curvature(wx, wy, ds))).tolist()[:10]

t_default = %timeit -o calculate_curvature(wx, wy, ds)
r_default = (1 / calculate_curvature(wx, wy, ds)).tolist()[:10]

In [43]:
print("\nVector test:\n{} \n{}".format(t_vector, r_vector))
print("\nSpline test:\n{} \n{}".format(t_spline, r_spline))
print("\nDefault test:\n{} \n{}".format(t_default, r_default))


Vector test:
600 µs ± 5.15 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each) 
[13.504176855476265, 6.3489550257814615, 4.951128048443533, 7.566267434586032, 18.371708777829113, 40.017861005926726, 15.867688114356195, 9.391545229469617, 9.864821440218503, 18.1204985021757]

Spline test:
761 µs ± 15.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each) 
[-1.036692339042675e+16, 13.489905239824152, 6.273348283154895, 4.583746334950832, 7.691486743511906, 18.825165073486, 52.42479601800344, 15.78212714221633, 9.109764520574096, 9.77077257041433]

Default test:
593 µs ± 3.17 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each) 
[13.778153793742078, 6.505713159974447, 4.937286760627668, 7.4139258953556615, 18.24968043695906, 40.45969693406867, 16.080061487876694, 9.451627047904548, 9.805865024357892, 17.979095861755038]


# Prepending

In [44]:
%%capture

def solve_1st_derivative_insert(x, y):

    dx = np.ediff1d(x)
    dy = np.ediff1d(y)
    dx = np.insert(dx, 0, dx[-1])
    dy = np.insert(dy, 0, dy[-1])

    return dx, dy

def solve_1st_derivative_concat(x, y):

    dx = np.ediff1d(x)
    dy = np.ediff1d(y)
    dx = np.concatenate((dx, [dx[-1]]))
    dy = np.concatenate((dy, [dy[-1]]))

    return dx, dy

t_insert = %timeit -o solve_1st_derivative_insert(wx, wy)
d = solve_1st_derivative_insert(wx, wy)
d_insert = d[0][:5]

t_concat = %timeit -o solve_1st_derivative_concat(wx, wy)
d = solve_1st_derivative_concat(wx, wy)
d_concat = d[0][:5]

In [45]:
print("\nInsert test:\n{}\n{}".format(t_insert, d_insert))
print("\nConcatenate test:\n{}\n{}".format(t_concat, d_concat))


Insert test:
25.5 µs ± 724 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
[8.0661     2.15463259 1.21198083 0.73237982 0.06881949]

Concatenate test:
6.33 µs ± 28.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
[2.15463259 1.21198083 0.73237982 0.06881949 0.05440933]


# Tuple

In [46]:
%%capture

def calculate_curvature_vector(wx, wy):

    dp = solve_1st_derivative(wx, wy)
    delta_s = np.hypot(dp[0], dp[1])
    u_i = dp / delta_s

    du = solve_1st_derivative(u_i[0], u_i[1])
    abs_du = np.hypot(du[0], du[1])
    k_i = abs_du / delta_s

    return k_i

def calculate_curvature_vector_tuple(wx, wy):

    dp_x, dp_y = solve_1st_derivative(wx, wy)
    delta_s = np.hypot(dp_x, dp_y)
    u_i_x = dp_x / delta_s
    u_i_y = dp_y / delta_s

    du_x, du_y = solve_1st_derivative(u_i_x, u_i_y)
    abs_du = np.hypot(du_x, du_y)
    k_i = abs_du / delta_s

    return k_i

t_tuple = %timeit -o calculate_curvature_vector_tuple(wx, wy)
t_default = %timeit -o calculate_curvature_vector(wx, wy)

In [47]:
print("\nTuple test:\n{}".format(t_tuple))
print("\nDefault test:\n{}".format(t_default))


Tuple test:
17.2 µs ± 119 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Default test:
18.7 µs ± 619 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [48]:
%%capture

def solve_1st_derivative_concat_tuple(s):

    ds = np.diff(s)
    ds = np.concatenate((ds, [ds[-1]]))

    return ds

t_tuple = %timeit -o solve_1st_derivative_concat_tuple((wx, wy))
d_tuple = solve_1st_derivative_concat_tuple((wx, wy))

t_default = %timeit -o solve_1st_derivative_concat(wx, wy)
d_default = solve_1st_derivative(wx, wy)

In [49]:
print("\nTuple test:\n{}\n{}".format(t_tuple, d_tuple))
print("\nDefault test:\n{}\n{}".format(t_default, d_default))


Tuple test:
7.51 µs ± 175 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
[[ 2.15463259  1.21198083  0.73237982  0.06881949  0.05440933 -0.03501278
  -0.28010224 -0.42015336  0.24518732  0.86481566  1.1658277   1.92570288
   1.78565176  2.26990153  2.66960401  3.66488103  3.47810295  7.44988714
   2.41022589  2.55862621  3.55514378  4.22846647  6.22150162  7.28912207
   7.6798462   7.4389      7.0931      7.9987      8.0661    ]
 [ 1.83143771  2.58555911  2.65531525  3.45369463  3.98562497  2.80102237
   2.5909457   2.62595847  2.82893839  2.50341375  1.6702038   1.50554953
   1.36549841  1.50321534  1.85624033  2.50516538  2.34855563  4.70054412
   1.30463004  0.7271885   0.61945687  0.43092652  0.70025559  0.76920384
   1.03028681  0.845       0.7592      0.8385      0.8935    ]
 [ 1.83143771  2.58555911  2.65531525  3.45369463  3.98562497  2.80102237
   2.5909457   2.62595847  2.82893839  2.50341375  1.6702038   1.50554953
   1.36549841  1.50321534  1.85624033  2.505165

# Concatenation

In [50]:
t_list = %timeit -o np.concatenate(([0], np.hypot(wx, wy)))
t_zeros = %timeit -o np.concatenate((np.zeros(1), np.hypot(wx, wy)))
t_array = %timeit -o np.concatenate((np.array([0]), np.hypot(wx, wy)))

2.22 µs ± 40.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.18 µs ± 32.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.43 µs ± 15.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [51]:
print(f"\n List test:\n {t_list}")
print(f"\n Zeros test:\n {t_zeros}")
print(f"\n Array test:\n {t_array}")


 List test:
 2.22 µs ± 40.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

 Zeros test:
 2.18 µs ± 32.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

 Array test:
 2.43 µs ± 15.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Numba

In [86]:
from scipy.spatial import cKDTree
from scipy.spatial.distance import cdist

def calculate_closest_id(x, y, point):

    dx = [point[0] - icx for icx in x]
    dy = [point[1] - icy for icy in y] 

    d = np.hypot(dx, dy)

    return np.min(d), np.argmin(d), dx, dy

def calculate_closest_id_numpy(x, y, point):

    dx = point[0] - x
    dy = point[1] - y

    d = np.hypot(dx, dy)

    return np.min(d), np.argmin(d), dx, dy

calculate_closest_id_numpy_jit = jit(nopython=True)(calculate_closest_id_numpy)

scipy_cdist = cdist(np.array([wx, wy]).T, np.array([[wx[-1], wy[-1]]])).argmin()
scipy_ckdtree = cKDTree(np.array([wx, wy]).T).query(np.array([wx[-1], wy[-1]]), k=1, workers=-1)
numba = calculate_closest_id_numpy_jit(wx, wy, (wx[-1], wy[-1]))
numpy = calculate_closest_id_numpy(wx, wy, (wx[-1], wy[-1]))
standard = calculate_closest_id(wx.tolist(), wy.tolist(), (float(wx[-1]), float(wy[-1])))

print(f"\nSciPy cDist test:\n {scipy_cdist}")
print(f"\nSciPy cKDTree test:\n {scipy_ckdtree}")
print(f"\nNumba test:\n {numba}")
print(f"\nNumpy test:\n {numpy}")
print(f"\nStandard test:\n {standard}")

29

SciPy test:
 (0.0, 29)

Numba test:
 (0.0, 29, array([93.5462479 , 91.39161531, 90.17963447, 89.44725466, 89.37843516,
       89.32402583, 89.35903861, 89.63914085, 90.05929421, 89.81410689,
       88.94929123, 87.78346353, 85.85776065, 84.07210888, 81.80220735,
       79.13260335, 75.46772232, 71.98961937, 64.53973222, 62.12950634,
       59.57088013, 56.01573635, 51.78726988, 45.56576826, 38.2766462 ,
       30.5968    , 23.1579    , 16.0648    ,  8.0661    ,  0.        ]), array([54.23503104, 52.40359334, 49.81803422, 47.16271898, 43.70902434,
       39.72339938, 36.922377  , 34.33143131, 31.70547283, 28.87653444,
       26.3731207 , 24.7029169 , 23.19736737, 21.83186897, 20.32865363,
       18.4724133 , 15.96724792, 13.61869229,  8.91814816,  7.61351813,
        6.88632963,  6.26687276,  5.83594624,  5.13569064,  4.36648681,
        3.3362    ,  2.4912    ,  1.732     ,  0.8935    ,  0.        ]))

Numpy test:
 (0.0, 29, array([93.5462479 , 91.39161531, 90.17963447, 89.44725466

In [92]:
scipy_cdist_t = %timeit -o cdist(np.array([wx, wy]).T, np.array([[wx[-1], wy[-1]]])).argmin()
scipy_ckdtree_t = %timeit -o cKDTree(np.array([wx, wy]).T).query(np.array([wx[-1], wy[-1]]), k=1, workers=-1)
numba_t = %timeit -o calculate_closest_id_numpy_jit(wx, wy, (wx[-1], wy[-1]))
numpy_t = %timeit -o calculate_closest_id_numpy(wx, wy, (wx[-1], wy[-1]))
standard_t = %timeit -o calculate_closest_id(wx.tolist(), wy.tolist(), (float(wx[-1]), float(wy[-1])))

12.8 µs ± 182 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
62.7 µs ± 570 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
2.04 µs ± 24.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
8.58 µs ± 135 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
30.9 µs ± 87.1 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [93]:
print(f"\nSciPy cDist test:\n {scipy_cdist_t}")
print(f"\nSciPy cKDTree test:\n {scipy_ckdtree_t}")
print(f"\nNumba test:\n {numba_t}")
print(f"\nNumpy test:\n {numpy_t}")
print(f"\nStandard test:\n {standard_t}")


SciPy cDist test:
 12.8 µs ± 182 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

SciPy cKDTree test:
 62.7 µs ± 570 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)

Numba test:
 2.04 µs ± 24.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Numpy test:
 8.58 µs ± 135 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Standard test:
 30.9 µs ± 87.1 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
